In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="sixty_north.css">

# `Hypothesis`
**Property-based testing for Python**

**TODO** Verify this!!!
[hypothesisworks.com](https://hypoethesisworks.com)

**"...the larger purpose of Hypothesis is to drag the world kicking and screaming into a new
and terrifying age of high quality software."**  

*--David MacIver, Hypothesis manifesto*

* Basic idea: express "hypotheses"/invariants about your code and let `hypothesis` try to disprove them. Fundamentally, it searches for sets input to your test functions which trigger assertions in the tests.
* Strategies: An object with methods that describe how to generate and simplify certain kinds of values.
  * Hypothesis simplifies when it finds breaking inputs
  * primitive types
  * lists, tuples
  * choice, sample_from
  * stream
  * mapping, filtering
  * flatmap
  * recursive
  * composite
  * draw (interactive strategy drawing)
* @given
  * How hypothesis mathes givens to arguments
* @example
* assume (nan example)
* settings
  * e.g. max_examples, verbosity
  * profiles
* note
* event
* test statistics
* executors
* find() for data exploration
* extras: django, datetime, numpy, etc.
* database
* stateful testing

In [1]:
%run unittest_magics.py
from hypothesis import given

# Basic concept

1. Define invariants or "hypotheses" about your code
2. Tell `hypothesis` how to parameterize your tests
3. `hypothesis` searches for parameters that disprove your invariants

# A motivating example

This simple GCD function will help us see the fundamental value of `hypothesis`.

In [2]:
def gcd(m, n):
    while m and n:
        m, n = max(m, n) - min(m, n), min(m, n)
    return max(m, n)

With example-based testing we'd throw a lot of examples at the function and look for errors.

In [2]:
%%unittest_run

import itertools

class Tests(unittest.TestCase):
    def test_zero_remainder(self):
        for m,n in itertools.product(range(1, 1000), range(1, 100)):
            d = gcd(m, n)
            self.assertEqual(m % d, 0)
            self.assertEqual(n % d, 0)
            
    def test_gcd_is_greatest(self):
        for m,n in itertools.product(range(1, 100), range(1, 100)):
            d = gcd(m, n)
            for other in range(d + 1, 100):
                self.assertTrue(m % other or n % other)

FAILURES: 0 ERRORS: 2 SKIPPED: 0 TOTAL: 2
e: test_gcd_is_greatest (__main__.Tests)
e: test_zero_remainder (__main__.Tests)


In [3]:
%%unittest_run

from hypothesis.strategies import assume, integers

class Tests(unittest.TestCase):
    @given(integers(min_value=1, max_value=2000), integers(min_value=1, max_value=2000))
    def test_zero_remainder(self, m, n):
        d = gcd(m, n)
        self.assertEqual(m % d, 0)
        self.assertEqual(n % d, 0)
            
    @given(integers(min_value=1, max_value=2000), integers(min_value=1, max_value=2000), integers(min_value=1, max_value=200))
    def test_gcd_is_greatest(self, m, n, other):
        d = gcd(m, n)
        assume(other > d)
        self.assertTrue(m % other or n % other)

Falsifying example: test_gcd_is_greatest(self=<__main__.Tests testMethod=test_gcd_is_greatest>, m=1, n=1, other=1)
Falsifying example: test_zero_remainder(self=<__main__.Tests testMethod=test_zero_remainder>, m=1, n=1)
FAILURES: 0 ERRORS: 2 SKIPPED: 0 TOTAL: 2
e: test_gcd_is_greatest (__main__.Tests)
e: test_zero_remainder (__main__.Tests)


# Strategies
*Define how to generate and simplify input data*

In [5]:
import hypothesis.strategies as st

strategies = [
    st.integers(),
    st.text(),
    st.fractions(),
    st.lists(st.booleans())
]

[s.example() for s in strategies]

[-72, '', Fraction(-4, 46131), [True, True, True]]

## Primitive strategies
*Generate all of the Python primitive types (and then some)*

* `floats`, `integers`, `booleans`, `None`
* `text`, `binary`
* `uuids`, `fractions`

You will often use these directly, as well as part of more complex strategies

In [6]:
%%unittest_run

from hypothesis import given

class Tests(unittest.TestCase):
    @given(st.uuids(), st.fractions(), st.binary(min_size=2, max_size=10))
    def test_primitive_strategies(self, uuid, fraction, binary):
        print(uuid, fraction, binary)
        self.assertEqual(uuid, fraction)

fbb4cec9-7132-903d-0133-567ccdd9b281 -183/815 b'DhS~\x99\xd2\t\x1fN\xe1'
fbb4cec9-7132-903d-0133-567ccdd9b281 -183/815 b'DhS~\x99\xd2\t\x1fN\xe1'
fbb4cec9-7132-903d-0133-567ccdd9b281 -183/815 b'DhS\xd2\t\x1fN\xe1'
e9274cad-67ea-3d60-a94c-dc542533874f 2710936726419280443029577633263255552/135389463797825537 b'DhS\xd2\t\x01'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'D\x01\x01'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'D\x01'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'D\x01'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'\x00\x01'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'\x00\x00'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752/74624959292498177 b'\x00\x00'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 2465582589519981814218752 b'\x00\x00'
5a00ba02-5828-e5b8-9f65-68039aadd2ba 0 b'\x00\x

## Container strategies
*Generate lists, dictionaries, and tuples using other strategies*

* `lists`
* `dictionaries`, `fixeddictionaries`
* `sets`, `frozensets`
* `streaming`
* `tuples`

In [7]:
%%unittest_run

class Tests(unittest.TestCase):
    @given(st.dictionaries(st.integers(), st.text()), st.tuples(st.floats()))
    def test_container_strategies(self, d, s):
        print(d, s)
        self.assertEqual(len(d), len(s))

{8: '\U000ab90b礤\U000812e5\U00044326Ñ', -32430: '\x98䜋'} (6.081201111164977e+18,)
{8: '\U000ab90b礤\U000812e5\U00044326Ñ', -32430: '\x98䜋'} (6.081201111164977e+18,)
{8: '\U000ab90b礤\U000812e5\U00044326Ñ', -32430: ''} (9.69580290841861e+192,)
{8: '\U000ab90b\U000812e5\U00044326Ñ', -32430: ''} (9.69580290841861e+192,)
{1993841993677373809355710590420516990: ''} (1.0682094739224106e-306,)
{8: '\U000ab90b\U000812e5\U00044326Ñ0'} (0.0,)
{1384738019747304446050676188125331617: '', -32430: ''} (9.69580290841861e+192,)
{1993841993677373809355710590420516990: ''} (1.0682094739224106e-306,)
{7788445287802241442795744493830144: ''} (5.64981402286857e-310,)
{-32430: ''} (9.69580290841861e+192,)
{7788445287802241442795744493830144: ''} (5.64981402286857e-310,)
{1384738019747304446050676188125331617: '0'} (0.0,)
{} (9.69580290841861e+192,)
{} (9.69580290841861e+192,)
{} (0.0,)
Falsifying example: test_container_strategies(self=<__main__.Tests testMethod=test_container_strategies>, d={}, s=(0.0,))
{} 

## Mapping and filtering strategies
*Declarative modification and constraint of strategies*

* `<any strategy>.map(func)`: Map a function over a strategy
* `<any strategy>.filter(func)`: Keep only matching values from a strategy

In [8]:
%%unittest_run

class Tests(unittest.TestCase):
    @given(st.integers().map(lambda x: x * 2), 
           st.integers().filter(lambda x: x % 2 == 0))
    def test_even_minus_even_is_odd(self, x, y):
        self.assertTrue((x - y) % 2 == 0)

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


## `<strategy>.flatmap()` 
*Use one example to generate further strategies*

In [9]:
s = st.integers(min_value=1, max_value=3).flatmap(
    lambda n: st.lists(st.lists(st.integers(), min_size=n, max_size=n)))
s.example()

[[-249, 119], [164, -53216]]

## Other strategy goodies

* `<strategy>.recursive()`: Generate recursive data structures, e.g. JSON data
* `hypothesis.strategies.composite`: Combine strategies in arbitrary ways, e.g. lists plus an index
* `hypothesis.strategies.data`: Interactively generate values from strategies

# The `given` decorator
*Defines how strategies are associated with test function parameters*

From the `hypothesis` documentation:

* You may pass any keyword argument to `given`.
* Positional arguments to `given` are equivalent to the rightmost named
  arguments for the test function.
* Positional arguments may not be used if the underlying test function has
  varargs or arbitrary keywords.
* Functions tested with `given` may not have any defaults.


In [10]:
# All of these tests are equivalent

class Tests(unittest.TestCase):
    @given(x=st.integers(), y=st.floats())
    def test_example1(self, x, y):
        print(x, y)
    
    @given(st.integers(), st.floats())
    def test_example2(self, x, y):
        print(x, y)
    
    @given(st.floats(), x=st.integers())
    def test_example3(self, x, y):
        print(x, y)

# The `example` decorator
*Instruct hypothesis to use some specific examples*

Examples will be executed before strategy-defined inputs.

Useful for testing known edge-cases or other trouble spots.

In [11]:
%%unittest_run

from hypothesis import example, settings
from string import ascii_letters

class Tests(unittest.TestCase):
    @given(st.text(alphabet=ascii_letters), st.text(alphabet=ascii_letters))
    @example('', '')
    @example('foo', 'bar')
    @settings(max_examples=1)
    def test_levenshtein(self, m, n):
        print('inputs:', m, n)

inputs:  
inputs: foo bar
inputs: L 
FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


# `assume()`
*Indicate that failures under certain conditions should be ignored*

`assume()` lets you tell `hypothesis` that the current examples should be considered "bad examples". The test will be aborted with an exception but `hypothesis` will not consider it a failure.

In [12]:
%%unittest_run

from hypothesis import given, assume
from math import isnan

# adapted from hypothesis documentation
class Tests(unittest.TestCase):
    @given(st.floats())
    def test_negation_is_self_inverse_for_non_nan(self, x):
        assume(not isnan(x))
        self.assertEqual(x, -(-x))

FAILURES: 0 ERRORS: 0 SKIPPED: 0 TOTAL: 1


# Stateful testing
*Find bad states by automatically driving your API*

* Define API call relationships with a state machine
* `hypothesis` can still help generate input data
* Use assertions and preconditions to check your program state

This is new and semi-public, but is considered ready for general use.

# Further topics

* `settings`: get and set values that control how `hypothesis` behaves
* `note`: additional diagnostics for final test runs
* `event`: inject messages into `hypothesis`'s event log
* executors: control the test execution context
* `find()`: explore example sets for those that meet certain criteria
* extras: django, datetime, numpy, etc.
* database: `hypothesis` stores/caches information in a directory structure